# Projeto

Você foi contratado por uma empresa de ração para cães para tentar prever por que alguns lotes de sua ração estão estragando muito mais rápido do que o planejado! Infelizmente, esta empresa de comida para cães não atualizou para as máquinas mais recentes, o que significa que as quantidades dos cinco produtos químicos conservantes que eles estão usando podem variar muito, mas qual é o produto químico que tem o efeito mais forte? A empresa de ração para cães primeiro mistura um lote de conservante que contém 4 conservantes químicos diferentes (A, B, C, D) e, em seguida, é completado com um produto químico de "enchimento". Os cientistas de alimentos acreditam que um dos conservantes A, B, C ou D está causando o problema, mas precisam da sua ajuda para descobrir qual deles!

Use o Aprendizado de Máquina com RF para descobrir qual parâmetro tem o poder mais preditivo, descobrindo assim qual produto químico causa a deterioração precoce! Portanto, crie um modelo e descubra como você pode decidir qual produto químico é o problema!

* Pres_A: Porcentagem de conservante A na mistura
* Pres_B: Porcentagem de conservante B na mistura
* Pres_C: Porcentagem de conservante C na mistura
* Pres_D: Porcentagem de conservante D na mistura
* Estragado: Etiqueta indicando se o lote de ração estava estragado ou não.
___

** Pense cuidadosamente sobre o que esse problema realmente está pedindo para você resolver. Embora usemos o aprendizado de máquina para resolver isso, não será com seu fluxo de trabalho de divisão de treinamento / teste típico. Se isso confundir você, pule para o código da solução ao longo do passo a passo! **____

In [1]:
# Exemplo de métodos de árvore
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dogfood').getOrCreate()

In [2]:
# Carregando dados de treino
data = spark.read.csv('dog_food.csv',inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [4]:
data.head()

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)

In [5]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [6]:
# Importando VectorAssembler e Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [8]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol="features")

In [9]:
output = assembler.transform(data)

In [10]:
from pyspark.ml.classification import RandomForestClassifier,DecisionTreeClassifier

In [11]:
rfc = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')

In [12]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
final_data = output.select('features','Spoiled')
final_data.head()

Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)

In [14]:
rfc_model = rfc.fit(final_data)

In [16]:
rfc_model.featureImportances

SparseVector(4, {1: 0.0019, 2: 0.9832, 3: 0.0149})


Bingo! A característica no índice 2 (Química C) é de longe a característica mais importante, o que significa que está causando a deterioração precoce! Este é um uso muito interessante de um modelo de aprendizado de máquina de uma forma alternativa!